In [1]:
!pip install --upgrade pythainlp
!pip install pyLDAvis

# import pyLDAvis.gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

Requirement already up-to-date: pythainlp in /usr/local/lib/python3.7/dist-packages (2.3.1)


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe.

In [2]:
import pandas as pd
import pythainlp
import gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
df = pd.read_csv('/content/drive/MyDrive/CRM/voice of cus/Wongnai Reviews - Small.csv')

     |████████████████████████████████| 256kB 4.9MB/s 
     |████████████████████████████████| 10.1MB 8.1MB/s 
     |████████████████████████████████| 81kB 9.3MB/s 
     |████████████████████████████████| 61kB 6.4MB/s 
     |████████████████████████████████| 3.1MB 30.5MB/s 
     |████████████████████████████████| 645kB 34.8MB/s 
     |████████████████████████████████| 1.1MB 34.8MB/s 
     |████████████████████████████████| 112kB 44.9MB/s 
     |████████████████████████████████| 296kB 41.9MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp37-none-any.whl size=27099 sha256=975774415454b9b541d836170695792346c46a3eafa1835dda8da203e2c0c75e
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for phik: filename=phik-0.11.2-cp37-none-any.whl size=1107437 sha256=043e53c5acba1a64e06bfba90795d530716d3e43c821bb70b9bad6539aed01e3
  Stored in directory: /root/.cache/pip/wheels/c0/a3/b0/f27b1cfe32ea131a3715169132ff6d8565

In [4]:
df.tail()


,Review ID,Review
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...


In [5]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = [' ', '  ', '\n', 'ร้าน', '(', ')']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [6]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [7]:
df.tail()

,Review ID,Review,Review_tokenized
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ..."
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย..."
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน..."
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค..."
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,..."


In [8]:
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [9]:
print(dictionary.token2id.keys())

dict_keys(['20', 'Macchiato', 'กาแฟ', 'กาแฟร้อน', 'กิน', 'คน', 'ครึ่ง', 'ความคิด', 'ชอบ', 'ดื่ม', 'ตอน', 'ทาน', 'นึง', 'บาท', 'ปริมาณ', 'มีความสุข', 'ลา', 'สั่ง', 'หนัก', 'หลังจากนี้', 'เข้าไป', 'เจอ', 'เดิน', 'เดิม', 'เป็นประจำ', 'เฟล', 'แก้ว', 'โบราณ', '-', 'Art', 'Coffee', 'Kasetsart', '^^', 'of', 'คาราเมล', 'คิ', 'ชา', 'ซ', 'ซี', 'ดี', 'นั่ง', 'น่ารัก', 'บรรยากาศ', 'พนักงาน', 'พูดจา', 'มัค', 'รสชาติ', 'ราคา', 'ร้านกาแฟ', 'ลอน', 'ลาเต้', 'ลูกค้า', 'อัธยาศัย', 'อัสสัม', 'อา', 'อิตาเลียน', 'อุดหนุน', 'เพรส', 'เมนู', 'เยี่ยม', 'เอ๊กซ์', 'แพง', 'โซดา', 'โซ่', 'โต้', 'ไอ', 'ไอซ์', '555', 'กก', 'กรอบ', 'กระ', 'กระทะ', 'กร๊อบ', 'กลิ่น', 'กวง', 'กะ', 'กินน้ำ', 'กุ้ง', 'ก้น', 'ข้าวผัด', 'ข้าวเหนียว', 'ความสนใจ', 'คอ', 'คาว', 'งั้น', 'งี้', 'จริ๊ง', 'จัดจ้าน', 'จาน', 'จี๊ดจ๊าด', 'จ้น', 'จ้ิม', 'จ๊อบ', 'ฉ่า', 'ชิม', 'ชุ่มคอ', 'ดาว', 'ตบท้าย', 'ตัก', 'ถูกใจ', 'ทอด', 'ทะเล', 'ทีหลัง', 'นานา', 'นุ่ม', 'น้ำ', 'น้ำปลา', 'บรรยาย', 'ปลา', 'ปลาหมึก', 'ปาก', 'ปู', 'ปูม้า', 'ผสม', 'ผัด', 'พง', 'พริก', '

In [10]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [28]:
num_topics = 10
chunksize = 3000 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 50
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 6.88 s, sys: 82.5 ms, total: 6.97 s
Wall time: 6.92 s


In [33]:
gensimvis.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.048662 -0.001355       1        1  30.694590
5      0.014625  0.060867       2        1  13.427470
4     -0.025492 -0.022306       3        1  11.298666
6      0.048850 -0.029640       4        1  10.878762
9      0.011151 -0.016768       5        1  10.130202
8      0.001961  0.012095       6        1   8.197224
7     -0.005623 -0.011948       7        1   7.686497
1      0.004542  0.004807       8        1   3.177410
2      0.000235  0.002271       9        1   2.288215
0     -0.001588  0.001978      10        1   2.220965, topic_info=      Term        Freq       Total Category  logprob  loglift
157  อร่อย  478.000000  478.000000  Default  30.0000  30.0000
28       -  261.000000  261.000000  Default  29.0000  29.0000
11     ทาน  387.000000  387.000000  Default  28.0000  28.0000
2     กาแฟ  297.000000  297.000000  Default  27.0000  27.0000
58    เมนู  293.000000  293.000000  Default  26.0000  26.0000
..     ...         ...         ...      ...      ...      ...
197    ใส่    2.408412  102.708782  Topic10  -5.7830   0.0543
463     ผม    2.679450  175.654705  Topic10  -5.6764  -0.3757
40    นั่ง    2.699506  190.638268  Topic10  -5.6689  -0.4501
509    ลอง    2.624252  168.031698  Topic10  -5.6972  -0.3521
732   นะคะ    2.478704  147.380702  Topic10  -5.7543  -0.2781

[760 rows x 6 columns], token_table=      Topic      Freq                                               Term
term                                                                    
3691      6  0.825556                                                   
3661      2  0.503040                                                 !)
318       1  0.214514                                                  "
318       2  0.214514                                                  "
318       3  0.088329                                                  "
...     ...       ...                                                ...
4651      6  0.651975                                                  •
4693      7  0.593940                                                 ••
3148      5  0.514275  …................................................
2129      8  0.739028                                                  ′
2130      8  0.738574                                                  ″

[1350 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 5, 7, 10, 9, 8, 2, 3, 1])

In [29]:
model.show_topic(1)

[('อร่อย', 0.0106446),
 ('บาท', 0.009728873),
 ('กาแฟ', 0.009438967),
 ('รสชาติ', 0.009114841),
 ('ทาน', 0.008995663),
 ('กิน', 0.008233268),
 ('-', 0.008193393),
 ('ดี', 0.008185655),
 ('เมนู', 0.0077393954),
 ('ราคา', 0.0069698393)]

In [30]:

df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [31]:
df.tail()

,Review ID,Review,Review_tokenized,topics,score
295,296,ค่ำนี้คุณเพื่อนอยากส้มตำ หมูเฮาเลยพากันลงมากิน...,"ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ...",9,0.998508
296,297,ร้านสะอาดดี ตกแต่งสวยงาม มีที่จอดรถ ราคาเมนูต่...,"สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย...",3,0.558546
297,298,เช้าๆ รีบๆ วิ่งมาเข่าห้องเรียนแทบไม่ทันแต่ต้อง...,"เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน...",4,0.994721
298,299,ร้านนี้เป็นร้านกาแฟเล็กๆ ข้างๆ ร้านๆ Happy Man...,"ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค...",3,0.096220
299,300,ทรูคอฟฟี่สาขาซีคอนอยู่ในศูนย์บริการของทรู ชั้น...,"ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,...",3,0.997792


In [32]:
df['Review_tokenized']

0      คน,ชอบ,ทาน,Macchiato,เป็นประจำ,นึง,เดิน,เข้าไป...
1      Art,of,Coffee,Kasetsart,ร้านกาแฟ,รสชาติ,เยี่ยม...
2      กวง,ทะเล,เผา,อาหารทะเล,เค้า,สด,เนื้อ,ปู,หวาน,ค...
3      มีโอกาส,ตื่น,เช้า,โอกาส,มาหา,อร่อย,ทาน,ตอนเช้า...
4      ชอบ,ทาน,กิน,อาหาร,เวียดนาม,บ้าน,ซอย,ราชครู,เข้...
                             ...                        
295    ค่ำ,เพื่อน,ส้มตำ,หมู,เฮา,ลงมา,กิน,ส้มตำ,ออฟฟิศ...
296    สะอาด,ดี,ตกแต่ง,สวยงาม,ที่จอดรถ,ราคา,เมนู,เทีย...
297    เช้า,รีบ,วิ่ง,เข่า,ห้องเรียน,แทบ,ต้องหา,ของกิน...
298    ร้านกาแฟ,Happy,Mango,อาทิตย์,นัด,เพื่อน,นั่ง,ค...
299    ทรู,คอ,ฟ,ฟี่,สาขา,ซีคอน,ศูนย์,บริการ,ทรู,ชั้น,...
Name: Review_tokenized, Length: 300, dtype: object